Data

In [2]:
from datasets import load_dataset

# Example loading the 'emotion' dataset
emotion_data = load_dataset("emotion")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [7]:
from datasets import load_dataset
data = load_dataset("rotten_tomatoes")

# This will now work correctly
print(data)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 17060
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2132
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2132
    })
})


In [8]:
data['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 17060
})

In [9]:
data['validation']

Dataset({
    features: ['text', 'label'],
    num_rows: 2132
})

In [10]:
data['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 2132
})

In [11]:
data['train'][0]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'label': 1}

In [12]:
data['train'][0,-1]

{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'things really get weird , though not particularly scary : the movie is all portent and no content .'],
 'label': [1, 0]}

Text Classification with Representation Models

Using a Task-specific Model

In [13]:
from transformers import pipeline

model="cardiffnlp/twitter-roberta-base-sentiment-latest"

pipe=pipeline(
    model=model,
    tokenizer=model,
    return_all_scores=True,
    device="cuda:0"
)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [14]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

y_pred=[]

for output in tqdm(pipe(KeyDataset(data['test'],'text')),total=len(data['test'])):
  negative_score=output[0]['score']
  positive_score=output[1]['score']
  y_pred.append(np.argmax([negative_score,positive_score]))


100%|██████████| 2132/2132 [00:28<00:00, 75.58it/s] 


In [15]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true,y_pred):
  performance=classification_report(y_true,y_pred,
                                    target_names=['Negative Review','Positive Review'])
  print(performance)

In [16]:
evaluate_performance(data['test']['label'],y_pred)

                 precision    recall  f1-score   support

Negative Review       0.81      0.69      0.75      1066
Positive Review       0.73      0.84      0.78      1066

       accuracy                           0.77      2132
      macro avg       0.77      0.77      0.77      2132
   weighted avg       0.77      0.77      0.77      2132



Classification tasks that leverage embeddings

Supervised Classification

In [17]:
from sentence_transformers import SentenceTransformer
model=SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

#convert text to embeddings

train_embeddings=model.encode(data['train']['text'],show_progress_bar=True)
test_embeddings=model.encode(data['test']['text'],show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/534 [00:00<?, ?it/s]

Batches:   0%|          | 0/67 [00:00<?, ?it/s]

In [18]:
train_embeddings.shape

(17060, 768)

In [19]:
test_embeddings.shape

(2132, 768)

In [20]:
from sklearn.linear_model import LogisticRegression


clf=LogisticRegression(random_state=42)
clf.fit(train_embeddings,data["train"]["label"])

LogisticRegression(random_state=42)

In [21]:
y_pred=clf.predict(test_embeddings)

In [22]:
evaluate_performance(data['test']['label'],y_pred)

                 precision    recall  f1-score   support

Negative Review       0.85      0.86      0.85      1066
Positive Review       0.86      0.84      0.85      1066

       accuracy                           0.85      2132
      macro avg       0.85      0.85      0.85      2132
   weighted avg       0.85      0.85      0.85      2132



What would happen if we would not use a classifier at all?

Instead, we can average the embeddings per class and apply cosine similariy to predict which classes match the documents best:

In [23]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity

df=pd.DataFrame(np.hstack([train_embeddings,np.array(data['train']['label']).reshape(-1,1)]))
average_target_embeddings=df.groupby(768).mean().values

sim_matrix=cosine_similarity(test_embeddings,average_target_embeddings)
y_pred=np.argmax(sim_matrix,axis=1)

evaluate_performance(data['test']['label'],y_pred)

                 precision    recall  f1-score   support

Negative Review       0.85      0.84      0.84      1066
Positive Review       0.84      0.85      0.84      1066

       accuracy                           0.84      2132
      macro avg       0.84      0.84      0.84      2132
   weighted avg       0.84      0.84      0.84      2132



Zero-shot classification

In [24]:
label_embeddings=model.encode(['Negative Review','Positive Review'])
sim_matrix=cosine_similarity(test_embeddings,label_embeddings)
y_pred=np.argmax(sim_matrix,axis=1)

evaluate_performance(data['test']['label'],y_pred)

                 precision    recall  f1-score   support

Negative Review       0.77      0.77      0.77      1066
Positive Review       0.77      0.77      0.77      1066

       accuracy                           0.77      2132
      macro avg       0.77      0.77      0.77      2132
   weighted avg       0.77      0.77      0.77      2132



Classification with Generative Models

Encoder-decoder models

In [25]:
pipe=pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    device="cuda:0"
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [26]:
prompt="Is the following sentence positive or negative?"
data = data.map(lambda example: {"t5": prompt + example['text']})
data

Map:   0%|          | 0/17060 [00:00<?, ? examples/s]

Map:   0%|          | 0/2132 [00:00<?, ? examples/s]

Map:   0%|          | 0/2132 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 17060
    })
    validation: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 2132
    })
    test: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 2132
    })
})

In [27]:
y_pred=[]

for output in tqdm(pipe(KeyDataset(data["test"], "t5")), total=len(data["test"])):
    text = output[0]["generated_text"]
    y_pred.append(0 if text == "negative" else 1)

100%|██████████| 2132/2132 [01:56<00:00, 18.31it/s]


In [28]:
evaluate_performance(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.83      0.84      0.83      1066
Positive Review       0.84      0.83      0.83      1066

       accuracy                           0.83      2132
      macro avg       0.83      0.83      0.83      2132
   weighted avg       0.83      0.83      0.83      2132

